In [ ]:
from qiskit.quantum_info import SparsePauliOp

def tfim_hamiltonian(n_qubits, J=1.0, h=1.0):
    # Create a Hamiltonian for the transverse-field Ising model (TFIM)
    # H(J, h) = -J ∑ Z_i Z_{i+1} - h ∑ X_i
    paulis, coeffs = [], []
    # ZZ interactions
    for i in range(n_qubits):
        z_str = ['I']*n_qubits
        z_str[i] = z_str[(i+1) % n_qubits] = 'Z'
        paulis.append(''.join(reversed(z_str)))
        coeffs.append(-J)
    # X fields
    for i in range(n_qubits):
        x_str = ['I']*n_qubits
        x_str[i] = 'X'
        paulis.append(''.join(reversed(x_str)))
        coeffs.append(-h)
    return SparsePauliOp(paulis, coeffs)


In [2]:
from qiskit.circuit.library import EfficientSU2
from scipy.optimize import minimize
from qiskit.primitives import StatevectorEstimator as Estimator
import numpy as np

def ground_state_circuit(n_qubits, J, h, reps=2, maxiter=1000):

    hamiltonian = tfim_hamiltonian(n_qubits, J, h)

    ansatz = EfficientSU2(n_qubits, reps=reps)

    estimator = Estimator()

    def objective(params):
        pub = (ansatz, hamiltonian, params)
        job = estimator.run([pub])
        return job.result()[0].data.evs

    x0 = np.random.uniform(0, 2 * np.pi, ansatz.num_parameters)

    result = minimize(objective, x0, method='COBYLA', options={'maxiter': maxiter})
    
    optimal_circuit = ansatz.assign_parameters(result.x)
    return optimal_circuit

In [3]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def qcnn(n_qubits, depth=3):
    qc = QuantumCircuit(n_qubits, 1)

    θ_conv = ParameterVector('θ', length=n_qubits * depth)
    φ_pool = ParameterVector('φ', length=(n_qubits//2) * depth)
    ω_out  = ParameterVector('ω', length=1)

    conv_ptr, pool_ptr = 0, 0
    active = list(range(n_qubits))

    for d in range(depth):

        # Convolution layer
        for q in active:
            qc.ry(θ_conv[conv_ptr], q)
            conv_ptr += 1

        for i in range(0, len(active)-1, 2):
            qc.cx(active[i], active[i+1])

        # Pooling layer
        even, odd = active[::2], active[1::2]
        for e, o in zip(even, odd):
            qc.cx(e, o)
            qc.ry(φ_pool[pool_ptr], o)
            pool_ptr += 1
            
        active = odd  # keep half of the qubits

    qc.ry(ω_out[0], active[0])
    qc.measure(active[0], 0)
    return qc


In [ ]:
from qiskit_aer import AerSimulator
from qiskit import transpile
import numpy as np

def train_qcnn(states, labels, qcnn,             
               *,
               epochs: int  = 100,
               shots: int   = 2048,
               a0: float    = 0.10,
               c0: float    = 0.10,
               seed: int    = 42,
               jeffreys: float = 0.5):


    np.random.seed(seed)
    sim = AerSimulator(seed_simulator=seed)

    #SPSA 

    theta = np.random.uniform(0, 2 * np.pi, qcnn.num_parameters)

    qcnn_comp      = transpile(qcnn,  sim)
    prep_compiled  = [transpile(st, sim) for st in states]

    for k in range(epochs):
        a_k = a0 / (k + 1) ** 0.602          
        c_k = c0 / (k + 1) ** 0.101          
        delta = np.random.choice([1, -1], size=theta.shape)

        theta_plus  = theta + c_k * delta
        theta_minus = theta - c_k * delta

        grad_est   = np.zeros_like(theta)
        epoch_loss = 0.0

        for prep_circ, y in zip(prep_compiled, labels):
            circ_plus  = prep_circ.compose(qcnn_comp, front=True).assign_parameters(theta_plus,  inplace=False)
            circ_minus = prep_circ.compose(qcnn_comp, front=True).assign_parameters(theta_minus, inplace=False)

            counts_plus  = sim.run(circ_plus,  shots=shots).result().get_counts()
            counts_minus = sim.run(circ_minus, shots=shots).result().get_counts()

            n0_plus  = counts_plus.get('0', 0)
            n0_minus = counts_minus.get('0', 0)

            p0_plus  = (n0_plus  + jeffreys) / (shots + 2 * jeffreys)
            p0_minus = (n0_minus + jeffreys) / (shots + 2 * jeffreys)

            loss_p = -(y * np.log(p0_plus)  + (1 - y) * np.log(1 - p0_plus))
            loss_m = -(y * np.log(p0_minus) + (1 - y) * np.log(1 - p0_minus))

            epoch_loss += loss_p + loss_m
            grad_est   += (loss_p - loss_m) / (2 * c_k) * delta

        theta -= a_k * np.sign(grad_est / len(labels))

        if k % 10 == 0:
            avg_loss = epoch_loss / (2 * len(labels))
            print(f"Epoch {k:3d} | loss {avg_loss:.4f}")

    return theta


In [ ]:
from qiskit_aer.primitives import Sampler as AerSampler

n_qubits     = 8
depth        = 3          
train_grid   = [(1.0, h) for h in np.linspace(0.1, 0.9, 5)] + [(0.2, h) for h in np.linspace(1.1, 1.9, 5)]
labels       = [0]*5 + [1]*5   


states = [ground_state_circuit(n_qubits, J, h) for (J, h) in train_grid]

model = qcnn(n_qubits, depth)

trained_params = train_qcnn(states, labels, model, epochs=200)

test_state = ground_state_circuit(n_qubits, 1.0, 0.5)
test_circ  = test_state.compose(model, front=True)


aer_sampler = AerSampler(backend_options={"method": "statevector" },run_options={"shots": 2048})

result = aer_sampler.run([test_circ], parameter_values=[trained_params]).result()

num_bits   = test_circ.num_clbits or test_circ.num_qubits
zero_bits  = "0" * num_bits
prob_order = result.quasi_dists[0].get(zero_bits, 0.0)

print("\nPredicted phase:", "Ordered" if prob_order >= 0.5 else "Disordered")

Epoch   0 | loss 1.1317
Epoch  10 | loss 0.7255
Epoch  20 | loss 0.7233
Epoch  30 | loss 0.6960
Epoch  40 | loss 0.7606
Epoch  50 | loss 0.7173
Epoch  60 | loss 0.6955
Epoch  70 | loss 0.7161
Epoch  80 | loss 0.7522
Epoch  90 | loss 0.6953
Epoch 100 | loss 0.7143
Epoch 110 | loss 0.6951
Epoch 120 | loss 0.6952
Epoch 130 | loss 0.6954
Epoch 140 | loss 0.7130
Epoch 150 | loss 0.7125
Epoch 160 | loss 0.7119
Epoch 170 | loss 0.7874
Epoch 180 | loss 0.6950
Epoch 190 | loss 0.7116

Predicted phase: Disordered
